In [1]:
import numpy as np
import pandas as pd
from pandas import pivot_table
import matplotlib.pyplot as plt
import json
from colour import Color

In [2]:
borig = pd.read_csv('2018clean3.csv', header=0, index_col=0, dtype={'MainSic': object})

print(borig.shape)
borig.head(2)

(10772, 29)


,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,ResponsiblePerson,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted,Post,Val,MainSic,MainSection
0,"BRYANSTON SCHOOL,INCORPORATED","Bryanston House,\r\r\r\r\nBlandford,\r\r\r\r\n...",00226143,85310,19.6,32.5,NaN,NaN,0.0,0.0,...,Michael McGovern (Chief Operating Officer),500 to 999,"""Bryanston School"",Incorporated",False,05/04/2019 00:00:00,03/04/2019 13:43:34,DT11 0PX,True,85310,Education
1,"RED BAND CHEMICAL COMPANY, LIMITED","19 Smith's Place,\r\r\r\r\nLeith Walk,\r\r\r\r...",SC016876,47730,7.7,-4.2,23.7,-75.2,22.2,82.0,...,Philip Galt (Managing Director),250 to 499,"""RED BAND"" CHEMICAL COMPANY, LIMITED",False,05/04/2019 00:00:00,01/04/2019 12:37:21,EH6 8NU,True,47730,Wholesale and retail trade; repair of motor ve...


In [3]:
addnuts = pd.read_csv('pc2018_uk_NUTS-2016_v2.0.csv', sep=';')

print(addnuts.shape)
addnuts.head()

(1759911, 2)


,NUTS3,CODE
0,'UKM73','EH10 7DP'
1,'UKM73','EH10 7DX'
2,'UKM73','EH10 7DY'
3,'UKM73','EH10 7DZ'
4,'UKM73','EH10 7EA'


In [4]:
def remove_quotes(x):
    return str(x)[1:-1]

In [5]:
addnuts['NUTS3'] = addnuts['NUTS3'].apply(remove_quotes)
addnuts['CODE'] = addnuts['CODE'].apply(remove_quotes)

addnuts.head(2)

,NUTS3,CODE
0,UKM73,EH10 7DP
1,UKM73,EH10 7DX


In [6]:
addnuts = addnuts[['CODE', 'NUTS3']]
addnuts.rename(columns={"CODE": "Post", "NUTS3": "Nuts3"}, inplace=True)

addnuts.head(2)

,Post,Nuts3
0,EH10 7DP,UKM73
1,EH10 7DX,UKM73


In [7]:
len(addnuts['Nuts3'].unique())

179

In [8]:
borig = pd.merge(borig, addnuts, on='Post')

borig.head(2)

,EmployerName,Address,CompanyNumber,SicCodes,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,MaleBonusPercent,FemaleBonusPercent,...,EmployerSize,CurrentName,SubmittedAfterTheDeadline,DueDate,DateSubmitted,Post,Val,MainSic,MainSection,Nuts3
0,"BRYANSTON SCHOOL,INCORPORATED","Bryanston House,\r\r\r\r\nBlandford,\r\r\r\r\n...",00226143,85310,19.6,32.5,NaN,NaN,0.0,0.0,...,500 to 999,"""Bryanston School"",Incorporated",False,05/04/2019 00:00:00,03/04/2019 13:43:34,DT11 0PX,True,85310,Education,UKK22
1,"RED BAND CHEMICAL COMPANY, LIMITED","19 Smith's Place,\r\r\r\r\nLeith Walk,\r\r\r\r...",SC016876,47730,7.7,-4.2,23.7,-75.2,22.2,82.0,...,250 to 499,"""RED BAND"" CHEMICAL COMPANY, LIMITED",False,05/04/2019 00:00:00,01/04/2019 12:37:21,EH6 8NU,True,47730,Wholesale and retail trade; repair of motor ve...,UKM75


In [9]:
n_piv = pivot_table(borig, values='DiffMeanHourlyPercent',
                     index=['Nuts3'], aggfunc=np.mean)


n_piv.rename(columns={'DiffMeanHourlyPercent':'Diff_avg'}, inplace=True)
n_piv.sort_values(by=['Diff_avg'], ascending=False, inplace=True)
n_piv['Diff_avg'] = round(n_piv['Diff_avg'], 2)
n_piv.reset_index(inplace=True)

print(n_piv.shape)

n_piv.head(2)

(172, 2)


,Nuts3,Diff_avg
0,UKN14,25.55
1,UKM61,24.30


In [10]:
nuts_name = pd.read_csv('NUTS_Level_3_January_2018_Ultra_Generalised_Clipped_Boundaries_in_the_United_Kingdom.csv')

print(nuts_name.shape)
nuts_name.head(2)

(179, 9)


,objectid,nuts318cd,nuts318nm,bng_e,bng_n,long,lat,st_areashape,st_lengthshape
0,1,UKC11,Hartlepool and Stockton-on-Tees,444952,522016,-1.30587,54.591351,3.014553e+08,108085.925509
1,2,UKC12,South Teesside,461718,519597,-1.04695,54.567848,3.027700e+08,86922.520900


In [11]:
nuts_name = nuts_name[['nuts318cd','nuts318nm','lat','long']]
nuts_name.rename(columns={'nuts318cd':'Nuts3','nuts318nm':'Name','long':'lon'}, inplace=True)

nuts_name.head()

,Nuts3,Name,lat,lon
0,UKC11,Hartlepool and Stockton-on-Tees,54.591351,-1.30587
1,UKC12,South Teesside,54.567848,-1.04695
2,UKC13,Darlington,54.535351,-1.56835
3,UKC14,Durham CC,54.685131,-1.84050
4,UKC21,Northumberland,55.300369,-2.07521


In [12]:
n_piv.tail()

,Nuts3,Diff_avg
167,UKM94,4.96
168,UKL14,3.03
169,UKN11,1.13
170,UKN07,-1.13
171,UKN15,-5.60


In [13]:
diff_list = [item for item in nuts_name.Nuts3.tolist() if item not in n_piv.Nuts3.tolist()]

diff_list

['UKM64', 'UKM65', 'UKM66', 'UKN08', 'UKN09', 'UKN10', 'UKN12']

In [14]:
all_piv = n_piv.copy()

all_piv.loc[172] = ['UKM64', 0]
all_piv.loc[173] = ['UKM65', 0]
all_piv.loc[174] = ['UKM66', 0]
all_piv.loc[175] = ['UKN08', 0]
all_piv.loc[176] = ['UKN09', 0]
all_piv.loc[177] = ['UKN10', 0]
all_piv.loc[178] = ['UKN12', 0]

all_piv.tail(10)

,Nuts3,Diff_avg
169,UKN11,1.13
170,UKN07,-1.13
171,UKN15,-5.60
172,UKM64,0.00
173,UKM65,0.00
174,UKM66,0.00
175,UKN08,0.00
176,UKN09,0.00
177,UKN10,0.00
178,UKN12,0.00


In [15]:
all_piv = pd.merge(all_piv, nuts_name, on='Nuts3')

all_piv.tail(5)

,Nuts3,Diff_avg,Name,lat,lon
174,UKM66,0.0,Shetland Islands,60.504951,-1.37344
175,UKN08,0.0,"Newry, Mourne and Down",54.149399,-6.08791
176,UKN09,0.0,Ards and North Down,54.563999,-5.64464
177,UKN10,0.0,Derry City and Strabane,54.808990,-7.41980
178,UKN12,0.0,Causeway Coast and Glens,55.039589,-6.59867


In [16]:
borig.Nuts3.value_counts()[borig.Nuts3.value_counts().between(0,9)]

UKM93    9
UKM81    9
UKK42    9
UKM78    9
UKL24    8
UKJ34    7
UKL12    5
UKN16    4
UKN07    3
UKM61    3
UKN11    3
UKN06    3
UKM63    2
UKN14    2
UKN13    1
UKL11    1
UKN15    1
Name: Nuts3, dtype: int64

In [17]:
small_sample_list = borig.Nuts3.value_counts()[borig.Nuts3.value_counts().between(0,9)].index.tolist()

In [18]:
for i in small_sample_list:
    
    all_piv.loc[all_piv.Nuts3 == i, 'Diff_avg'] = 0
    
all_piv.sort_values(by=['Diff_avg'], ascending=False, inplace=True)
all_piv.reset_index(drop=True, inplace=True)

all_piv.tail(2)

,Nuts3,Diff_avg,Name,lat,lon
177,UKN13,0.0,Antrim and Newtownabbey,54.693790,-6.17661
178,UKN12,0.0,Causeway Coast and Glens,55.039589,-6.59867


In [19]:
def layer_text(x):
    if x == 0:
        return 'No Data'
    else:
        return '{}%'.format(int(round(x, 0)))

In [20]:
all_piv['Text'] = all_piv['Diff_avg'].apply(layer_text)
all_piv['Rank'] = round(all_piv['Diff_avg'], 0).rank(method='dense').astype(int)

all_piv.tail(2)

,Nuts3,Diff_avg,Name,lat,lon,Text,Rank
177,UKN13,0.0,Antrim and Newtownabbey,54.693790,-6.17661,No Data,1
178,UKN12,0.0,Causeway Coast and Glens,55.039589,-6.59867,No Data,1


In [46]:
# all_piv.to_csv('2018nuts3allukavg.csv')

## Company Min Max

In [27]:
bedit = borig.copy()

bedit.shape

(10590, 30)

In [28]:
bedit = bedit[['Nuts3', 'EmployerName', 'DiffMeanHourlyPercent']]
bedit.rename(columns={'DiffMeanHourlyPercent':'Diff_avg'}, inplace=True)
bedit.reset_index(drop=True, inplace=True)

bedit.head(2)

,Nuts3,EmployerName,Diff_avg
0,UKK22,"BRYANSTON SCHOOL,INCORPORATED",19.6
1,UKM75,"RED BAND CHEMICAL COMPANY, LIMITED",7.7


In [30]:
bedit.head()

,Nuts3,EmployerName,Diff_avg
0,UKK22,"BRYANSTON SCHOOL,INCORPORATED",19.6
1,UKM75,"RED BAND CHEMICAL COMPANY, LIMITED",7.7
2,UKL22,118 LIMITED,7.7
3,UKE42,123 EMPLOYEES LTD,30.0
4,UKJ25,1509 GROUP,13.4


In [31]:
len(bedit.Nuts3.unique())

172

In [32]:
bmm = pd.DataFrame(columns=bedit.columns)

for i in bedit.Nuts3.unique():
    if i not in small_sample_list:
        bsample = bedit[bedit.Nuts3 == i]

        bmm = bmm.append(bsample[bsample.Diff_avg == bsample.Diff_avg.max()].iloc[0,:], ignore_index=True)
        bmm = bmm.append(bsample[bsample.Diff_avg == bsample.Diff_avg.min()].iloc[0,:], ignore_index=True)
    
bmm.reset_index(drop=True, inplace=True)

print(bmm.shape)
bmm.head()

(310, 3)


,Nuts3,EmployerName,Diff_avg
0,UKK22,WIMBORNE ACADEMY TRUST,38.7
1,UKK22,CLIPPER CONTRACTING GROUP LTD,-8.6
2,UKM75,SCOTTISH RUGBY UNION LIMITED,50.0
3,UKM75,THE SCOTTISH SALMON COMPANY LIMITED,-30.0
4,UKL22,CARDIFF CITY FOOTBALL CLUB LIMITED,76.8


In [36]:
bmm.tail(2)

,Nuts3,EmployerName,Diff_avg
308,UKM62,GLOBAL ENERGY (GROUP) LIMITED,39.0
309,UKM62,G & A BARNIE GROUP LIMITED,-9.9


In [38]:
starter = 310
for i in (small_sample_list + diff_list):
    bmm.loc[starter] = [i, 'N/A', 'N/A']
    starter +=1
    
bmm.tail()

,Nuts3,EmployerName,Diff_avg
329,UKM66,N/A,N/A
330,UKN08,N/A,N/A
331,UKN09,N/A,N/A
332,UKN10,N/A,N/A
333,UKN12,N/A,N/A


In [39]:
# bmm.to_csv('2018allukminmax.csv')

In [44]:
missing_coat = all_piv[(all_piv['Nuts3'].str.contains('UKL')) | (all_piv['Nuts3'].str.contains('UKM')) | (all_piv['Nuts3'].str.contains('UKN'))].Nuts3.tolist()

In [45]:
missing_coat =  list(set([item[:-1] for item in missing_coat]))

missing_coat

['UKL2', 'UKM5', 'UKM8', 'UKN1', 'UKL1', 'UKM7', 'UKM6', 'UKN0', 'UKM9']